In [45]:
import plotly.figure_factory as ff
import os
from datetime import timedelta, datetime
import pandas as pd 

In [163]:
os.getcwd()

'/Users/markgiannini/Desktop/7PMark/Ad Hoc Projects/10 28 19 - DAG Timing'

In [164]:
path = '/Users/markgiannini/Desktop/7PMark/Ad Hoc Projects/10 28 19 - DAG Timing'
os.chdir(path)

In [165]:
#dag_task_RI = pd.read_csv('dag_timing_info.csv')
dag_task_RI = pd.read_csv('dag_timing_info_full.csv')

In [166]:
#checking to make sure all necessary dags are included
len(dag_task_RI.dag_id.unique())

25

In [167]:
#rename columns
dag_tasks = dag_task_RI.rename(columns={"dag_id": "Task", "execution_date": "Start", 'sum(duration)': 'duration'})
dag_tasks['Start']= pd.to_datetime(dag_tasks['Start'])
dag_tasks['Start'] = dag_tasks['Start']
dag_tasks['duration'] = pd.to_timedelta(dag_tasks['duration'], unit = 'S')
dag_tasks.dtypes

Task                     object
Start       datetime64[ns, UTC]
duration        timedelta64[ns]
dtype: object

In [168]:
finish = dag_tasks.Start + dag_tasks.duration
dag_tasks['Finish'] = finish.dt.strftime("%Y-%m-%d %H:%M:%S") 
dag_tasks['Start'] = dag_tasks.Start.dt.strftime("%Y-%m-%d %H:%M:%S")
dag_tasks['Resource'] = 'Task Running'


In [183]:
#Checking individual dag runs to determine if they can be eliminated from graph
dag_tasks[dag_tasks.Task == 'Alvin-NFLX-Lag'] # Max time = 10 sec - eliminate
dag_tasks[dag_tasks.Task == 'Bart-DailyProcess'] #Max time = 24 sec - eliminate
dag_tasks[dag_tasks.Task == 'Email-Distro-Report-List'] #max time = 8 sec - eliminate

,Task,Start,duration,Finish,Resource
636,Email-Distro-Report-List,2019-10-20 12:00:00,00:00:01.004040,2019-10-20 12:00:01,Task Running


In [179]:
removal_cond1 = (dag_tasks.Start < '2019-10-25')
removal_cond2 = (dag_tasks.Task != 'CompanyMetrics-LoadTrigger')
removal_cond3 = (dag_tasks.Task != 'CompanyMetrics-Load')
removal_cond4 = (dag_tasks.Task != 'CompanyMetrics-MeltTrigger')
removal_cond5 = (dag_tasks.Task != 'CompanyMetrics-Melt')
removal_cond6 = (dag_tasks.Task != 'Dag-Failure-Log')
removal_cond7 = (dag_tasks.Task != 'Analyst-Permissions-Flush')
removal_cond8 = (dag_tasks.Task != 'Alvin-NFLX-Lag')
removal_cond9 = (dag_tasks.Task != 'Bart-DailyProcess')
removal_cond10 = (dag_tasks.Task != 'Email-Distro-Report-List')

dag_plotly = dag_tasks[removal_cond1 & removal_cond2 & removal_cond3 & removal_cond4 & removal_cond5 & removal_cond6 & removal_cond7 & removal_cond8 & removal_cond9 & removal_cond10].to_dict('records')

#Notes:
    #Company Melt & Load are on demand tasks - remove from chart for now
    #Removing Dag_Failure_Log since it only runs upon Errors
    #Cutting tasks starts at 10/25 to get full week only

In [185]:
colors = {'Task Running': 'rgb(220, 0, 0)'}
fig = ff.create_gantt(dag_plotly, colors = colors, index_col='Resource', show_colorbar=True,
                      group_tasks=True, bar_width=0.4, showgrid_x=True, showgrid_y=True)

fig.show()